In [1]:
# liberaries
#=================
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import datetime as dt
import time
from pymongo import MongoClient
import pymongo 

In [2]:
def connect_db():
    client = MongoClient("localhost", 27017)
    return client

client = connect_db()
mars = client.mars

In [3]:
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

# Create an instance of Flask
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb+srv://amr:<mars_app>@cluster0.c9c0y.mongodb.net/<mars>?retryWrites=true&w=majority"
mongo = PyMongo(app)

In [4]:
# excutable path 
#====================
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [5]:
# 1- Mars Latest news from NASA [ Title & Paragraphs]
#=======================================================================
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
time.sleep(1)

news_html = browser.html
news_soup = bs(news_html, "html.parser")
news_list = news_soup.find('ul', class_='item_list')
headline = news_list.find('div', class_="content_title").text
articles = news_list.find("div", class_="article_teaser_body").text
news_title = headline.strip()
news_p = articles.strip()

news_title
news_p

"The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight."

In [6]:
# 2- Mars Featured Image from JPL 
#=======================================================================
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')
image_url = jpl_soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image_url

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24247-640x350.jpg'

In [7]:
# 3- Mars Facts:
#=======================================================================
facts_url = "https://space-facts.com/mars/"
facts_df = pd.read_html(facts_url)
df = pd.DataFrame(facts_df[0])
df.columns = ["Planet Profile", "Mars"]
df.set_index("Planet Profile")

,Mars
Planet Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [8]:
mars_facts = df.to_html(index = True, header =True)

In [9]:
# 4- Mars Hemispheres:
#=======================================================================
hemisphere_image_urls = {}
page_link = "https://astrogeology.usgs.gov/"

hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
hemispheres_html = browser.html
## item_div = hemispheres_soup.find("div", class_ = "result-list")
hemispheres_soup = bs(hemispheres_html, "html.parser")
#================================================
hemisphere_div = hemispheres_soup.find_all("div", class_ ="item")

for hemisphere in hemisphere_div:
    titles = hemisphere.find("h3").text
titles

'Valles Marineris Hemisphere Enhanced'

In [10]:
 #Mars Hemispheres
hemisphere_image_urls=[]
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
hemispheres_html = browser.html
hemispheres_soup = bs(hemispheres_html, 'html.parser')
h3_div = hemispheres_soup.find_all("h3")
for title in h3_div:
    browser.links.find_by_partial_text("Hemisphere")
    discription_div = hemispheres_soup.find_all("div", class_="description")
    for div in discription_div:
        div_a = div.find('a')
        img_href = div_a['href']
        title_img = div_a.find('h3').text
        image_url = "https://astrogeology.usgs.gov" + img_href
        browser.visit(image_url)
        download_html = browser.html
        download_soup = bs(download_html, 'html.parser')
        full_img = download_soup.find("a", target="_blank")
        full_image = full_img['href']
        hemisphere_image_urls.append({"title":title_img,"img_url":full_image})






In [35]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://www.usgs.gov/centers/astrogeo-sc'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url':

In [11]:
# Mars Data Collection 
#==========================================================
mars_data = {
    "news_title": news_title,
    "news_p": news_p,
    "mars_facts": mars_facts,
    "jpl_image": featured_image_url,
    "mars_hemisphere": hemisphere_image_urls
}